In [1]:
import torch
import numpy as np
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

### model load

In [2]:
MODEL_PATH ='./configs/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py'

MODEL = 'vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco'
NAME = 'weighted-loss'
WORK_DIR = NAME+'_'+MODEL
# config file 들고오기
cfg = Config.fromfile(MODEL_PATH)

In [3]:
def print_keys(arg_dict):
    print(list(arg_dict.keys()))

print_keys(cfg)

['dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'model']


In [4]:
cfg.runner.max_epochs=40

### Dataset config

In [5]:
dataset_type = 'CocoDataset'
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
num_classes = len(classes)

root='../dataset/'

### dataset pipeline config
- augmentation

In [6]:
cfg.data.samples_per_gpu = 16
cfg.data.workers_per_gpu = 4

cfg.data.train.classes= classes
cfg.data.train.ann_file = root+'kfold/cv_train1.json'
cfg.data.train.img_prefix = root


cfg.data.val.classes = classes
cfg.data.val.ann_file = root+'kfold/cv_val1.json'
cfg.data.val.img_prefix = root

cfg.data.test.classes = classes
cfg.data.test.ann_file = root+'test.json'
cfg.data.test.img_prefix = root

cfg.data.train.pipeline[2].img_scale = [(1024, 512), (1024, 800)]
cfg.data.val.pipeline[1].img_scale = [(1024, 512), (1024, 800)]
cfg.data.test.pipeline[1].img_scale = [(1024, 512), (1024, 800)]
# # for TTA
cfg.data.test.pipeline[1].flip=True
cfg.data.val.pipeline[1].flip=True

# cfg.data.test.test_mode = True
cfg.data.val.test_mode = True

cfg.data.train = dict(
        type='ClassBalancedDataset',
        oversample_thr=1e-3,
        dataset= cfg.data.train)
cfg.data

{'samples_per_gpu': 16,
 'workers_per_gpu': 4,
 'train': {'type': 'ClassBalancedDataset',
  'oversample_thr': 0.001,
  'dataset': {'type': 'CocoDataset',
   'ann_file': '../dataset/kfold/cv_train1.json',
   'img_prefix': '../dataset/',
   'pipeline': [{'type': 'LoadImageFromFile'},
    {'type': 'LoadAnnotations', 'with_bbox': True},
    {'type': 'Resize',
     'img_scale': [(1024, 512), (1024, 800)],
     'multiscale_mode': 'range',
     'keep_ratio': True},
    {'type': 'RandomFlip', 'flip_ratio': 0.5},
    {'type': 'Normalize',
     'mean': [123.675, 116.28, 103.53],
     'std': [58.395, 57.12, 57.375],
     'to_rgb': True},
    {'type': 'Pad', 'size_divisor': 32},
    {'type': 'DefaultFormatBundle'},
    {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}],
   'classes': ('General trash',
    'Paper',
    'Paper pack',
    'Metal',
    'Glass',
    'Plastic',
    'Styrofoam',
    'Plastic bag',
    'Battery',
    'Clothing')}},
 'val': {'type': 'CocoDataset',
  'ann_file'

In [7]:
# Build dataset
datasets = build_dataset(cfg.data.train)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


### model config

In [8]:
cfg.model

{'type': 'VFNet',
 'backbone': {'type': 'ResNet',
  'depth': 50,
  'num_stages': 4,
  'out_indices': (0, 1, 2, 3),
  'frozen_stages': 1,
  'norm_cfg': {'type': 'BN', 'requires_grad': True},
  'norm_eval': True,
  'style': 'pytorch',
  'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'},
  'dcn': {'type': 'DCNv2', 'deform_groups': 1, 'fallback_on_stride': False},
  'stage_with_dcn': (False, True, True, True)},
 'neck': {'type': 'FPN',
  'in_channels': [256, 512, 1024, 2048],
  'out_channels': 256,
  'start_level': 1,
  'add_extra_convs': 'on_output',
  'num_outs': 5,
  'relu_before_extra_convs': True},
 'bbox_head': {'type': 'VFNetHead',
  'num_classes': 80,
  'in_channels': 256,
  'stacked_convs': 3,
  'feat_channels': 256,
  'strides': [8, 16, 32, 64, 128],
  'center_sampling': False,
  'dcn_on_last_conv': True,
  'use_atss': True,
  'use_vfl': True,
  'loss_cls': {'type': 'VarifocalLoss',
   'use_sigmoid': True,
   'alpha': 0.75,
   'gamma': 2.0,
   'iou_weight

In [9]:
print_keys(cfg.model.bbox_head)

['type', 'num_classes', 'in_channels', 'stacked_convs', 'feat_channels', 'strides', 'center_sampling', 'dcn_on_last_conv', 'use_atss', 'use_vfl', 'loss_cls', 'loss_bbox', 'loss_bbox_refine']


In [10]:
cfg.model.bbox_head.num_classes = num_classes

In [11]:
# weighted loss config
# 분포 구하기
instance_count = np.zeros(num_classes).astype(int)
for idx in range(len(datasets)):
    label = datasets.dataset.get_ann_info(idx)['labels']
    unique, counts = np.unique(label, return_counts=True)
    
    # add the occurrence number to each class
    instance_count[unique] += counts

    
normedWeights = [1 - (x / sum(instance_count)) for x in instance_count]
normedWeights = torch.FloatTensor(normedWeights).cuda()

## loss에 등록
cfg.model.bbox_head.loss_cls.loss_weight = normedWeights

### optimizer config

In [12]:
cfg.optimizer

{'type': 'SGD',
 'lr': 0.01,
 'momentum': 0.9,
 'weight_decay': 0.0001,
 'paramwise_cfg': {'bias_lr_mult': 2.0, 'bias_decay_mult': 0.0}}

In [13]:
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
print(cfg.optimizer)
print(cfg.optimizer_config)

{'type': 'SGD', 'lr': 0.01, 'momentum': 0.9, 'weight_decay': 0.0001, 'paramwise_cfg': {'bias_lr_mult': 2.0, 'bias_decay_mult': 0.0}}
{'grad_clip': {'max_norm': 35, 'norm_type': 2}}


### other config

In [14]:
cfg.checkpoint_config = dict(interval=1)

In [15]:
print(cfg.model.test_cfg)

{'nms_pre': 1000, 'min_bbox_size': 0, 'score_thr': 0.05, 'nms': {'type': 'nms', 'iou_threshold': 0.6}, 'max_per_img': 100}


In [16]:
cfg.model.test_cfg.score_thr =0
cfg.model.test_cfg.nms.type='soft_nms'

In [17]:
cfg.seed = 1010
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/{WORK_DIR}'

In [18]:
cfg.evaluation = dict(
    save_best='bbox_mAP',
    metric=['bbox']
)

In [19]:
# wandb
log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(
                project='p-stage_object_detection',
                name=f'{WORK_DIR}'))])
cfg.log_config = log_config

### config workflow

In [20]:
#cfg.workflow = [('train',1),('val',1)]

### save config for test

In [21]:
import pickle
import os
with open(os.path.join(cfg.work_dir,'cfg.pkl'),'wb') as f:
    pickle.dump(cfg,f)

## train

In [24]:
cfg.model.neck.type='PAFPN'

In [27]:
cfg.model

{'type': 'VFNet',
 'backbone': {'type': 'ResNet',
  'depth': 50,
  'num_stages': 4,
  'out_indices': (0, 1, 2, 3),
  'frozen_stages': 1,
  'norm_cfg': {'type': 'BN', 'requires_grad': True},
  'norm_eval': True,
  'style': 'pytorch',
  'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'},
  'dcn': {'type': 'DCNv2', 'deform_groups': 1},
  'stage_with_dcn': (False, True, True, True)},
 'neck': {'type': 'PAFPN',
  'in_channels': [256, 512, 1024, 2048],
  'out_channels': 256,
  'start_level': 1,
  'add_extra_convs': 'on_output',
  'num_outs': 5,
  'relu_before_extra_convs': True},
 'bbox_head': {'type': 'VFNetHead',
  'num_classes': 10,
  'in_channels': 256,
  'stacked_convs': 3,
  'feat_channels': 256,
  'strides': [8, 16, 32, 64, 128],
  'center_sampling': False,
  'dcn_on_last_conv': True,
  'use_atss': True,
  'use_vfl': True,
  'loss_cls': {'type': 'VarifocalLoss',
   'use_sigmoid': True,
   'alpha': 0.75,
   'gamma': 2.0,
   'iou_weighted': True,
   'loss_weight'

In [26]:
# Build the detector
model = build_detector(cfg.model)
model.init_weights()

/opt/ml/detection/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
2021-09-30 02:31:45,822 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2021-09-30 02:31:45,823 - mmcv - INFO - load model from: torchvision://resnet50
2021-09-30 02:31:45,824 - mmcv - INFO - Use load_from_torchvision loader
2021-09-30 02:31:46,227 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offset.bias, layer3.0.conv2.

2021-09-30 02:31:46,839 - mmcv - INFO - 
backbone.layer2.0.bn3.weight - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,840 - mmcv - INFO - 
backbone.layer2.0.bn3.bias - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,840 - mmcv - INFO - 
backbone.layer2.0.downsample.0.weight - torch.Size([512, 256, 1, 1]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,841 - mmcv - INFO - 
backbone.layer2.0.downsample.1.weight - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,841 - mmcv - INFO - 
backbone.layer2.0.downsample.1.bias - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,842 - mmcv - INFO - 
backbone.layer2.1.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,842 - mmcv - INFO - 
backbone.layer2.1.bn1.weight - torch.Size([128])

2021-09-30 02:31:46,861 - mmcv - INFO - 
backbone.layer3.1.bn1.bias - torch.Size([256]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,861 - mmcv - INFO - 
backbone.layer3.1.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,862 - mmcv - INFO - 
backbone.layer3.1.conv2.conv_offset.weight - torch.Size([27, 256, 3, 3]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-09-30 02:31:46,862 - mmcv - INFO - 
backbone.layer3.1.conv2.conv_offset.bias - torch.Size([27]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-09-30 02:31:46,863 - mmcv - INFO - 
backbone.layer3.1.bn2.weight - torch.Size([256]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,863 - mmcv - INFO - 
backbone.layer3.1.bn2.bias - torch.Size([256]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,863 - mmcv - INFO - 
backb

2021-09-30 02:31:46,882 - mmcv - INFO - 
backbone.layer4.0.conv1.weight - torch.Size([512, 1024, 1, 1]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,882 - mmcv - INFO - 
backbone.layer4.0.bn1.weight - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,882 - mmcv - INFO - 
backbone.layer4.0.bn1.bias - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,883 - mmcv - INFO - 
backbone.layer4.0.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-30 02:31:46,883 - mmcv - INFO - 
backbone.layer4.0.conv2.conv_offset.weight - torch.Size([27, 512, 3, 3]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-09-30 02:31:46,884 - mmcv - INFO - 
backbone.layer4.0.conv2.conv_offset.bias - torch.Size([27]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-09-30 02:31:46,884 - mmcv

2021-09-30 02:31:46,901 - mmcv - INFO - 
neck.downsample_convs.0.conv.bias - torch.Size([256]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-09-30 02:31:46,902 - mmcv - INFO - 
neck.downsample_convs.1.conv.weight - torch.Size([256, 256, 3, 3]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
2021-09-30 02:31:46,902 - mmcv - INFO - 
neck.downsample_convs.1.conv.bias - torch.Size([256]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-09-30 02:31:46,903 - mmcv - INFO - 
neck.pafpn_convs.0.conv.weight - torch.Size([256, 256, 3, 3]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
2021-09-30 02:31:46,903 - mmcv - INFO - 
neck.pafpn_convs.0.conv.bias - torch.Size([256]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-09-30 02:31:46,904 - mmcv - INFO - 
neck.pafpn_convs.1.conv.weight - torch.Size([256, 256, 3, 3]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
2021-09-30 02

In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2021-09-30 00:56:37,155 - mmdet - INFO - Start running, host: root@2f4df6ced18c, work_dir: /opt/ml/detection/mmdetection/work_dirs/weighted-loss_vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco
2021-09-30 00:56:37,156 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) 

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:192: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')
wandb: Currently logged in as: larcane (use `wandb login --relogin` to force relogin)


/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '
2021-09-30 00:57:07,720 - mmdet - INFO - Epoch [1][10/245]	lr: 1.162e-03, eta: 6:31:45, time: 2.401, data_time: 0.352, memory: 14790, loss_cls: 0.6181, loss_bbox: 0.9016, loss_bbox_rf: 1.2621, loss: 2.7818, grad_norm: 13.3939
2021-09-30 00:57:27,781 - mmdet - INFO - Epoch [1][20/245]	lr: 1.342e-03, eta: 5:59:11, time: 2.006, data_time: 0.038, memory: 14790, loss_cls: 0.9108, loss_bbox: 0.7865, loss_bbox_rf: 1.0700, loss: 2.7673, grad_norm: 9.2715
2021-09-30 00:57:47,674 - mmdet - INFO - Epoch [1][30/245]	lr: 1.522e-03, eta: 5:47:10

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 966/966, 0.8 task/s, elapsed: 1223s, ETA:     0s

...

2021-09-30 01:56:44,704 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_2.pth.
2021-09-30 01:56:44,705 - mmdet - INFO - Best bbox_mAP is 0.0230 at 2 epoch.
2021-09-30 01:56:44,710 - mmdet - INFO - Epoch(val) [2][966]	bbox_mAP: 0.0230, bbox_mAP_50: 0.0420, bbox_mAP_75: 0.0230, bbox_mAP_s: 0.0000, bbox_mAP_m: 0.0020, bbox_mAP_l: 0.0280, bbox_mAP_copypaste: 0.023 0.042 0.023 0.000 0.002 0.028
2021-09-30 01:57:07,693 - mmdet - INFO - Epoch [3][10/245]	lr: 9.982e-03, eta: 5:03:41, time: 2.296, data_time: 0.339, memory: 14790, loss_cls: 1.0457, loss_bbox: 0.4505, loss_bbox_rf: 0.5878, loss: 2.0840, grad_norm: 4.2009
2021-09-30 01:57:27,751 - mmdet - INFO - Epoch [3][20/245]	lr: 1.000e-02, eta: 5:03:30, time: 2.006, data_time: 0.037, memory: 14790, loss_cls: 1.0330, loss_bbox: 0.4459, loss_bbox_rf: 0.5795, loss: 2.0584, grad_norm: 3.8878
2021-09-30 01:57:47,782 - mmdet - INFO - Epoch [3][30/245]	lr: 1.000e-02, eta: 5:03:18, time: 2.003, data_time: 0.035, memory: 14790, lo

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 966/966, 0.9 task/s, elapsed: 1120s, ETA:     0s

2021-09-30 02:23:38,918 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.38s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.17s).
Accumulating evaluation results...
DONE (t=3.76s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.093
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.063
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.071
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.381
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.381
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.381
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxD

2021-09-30 02:23:50,579 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_3.pth.
2021-09-30 02:23:50,580 - mmdet - INFO - Best bbox_mAP is 0.0580 at 3 epoch.
2021-09-30 02:23:50,588 - mmdet - INFO - Epoch(val) [3][966]	bbox_mAP: 0.0580, bbox_mAP_50: 0.0930, bbox_mAP_75: 0.0630, bbox_mAP_s: 0.0000, bbox_mAP_m: 0.0090, bbox_mAP_l: 0.0710, bbox_mAP_copypaste: 0.058 0.093 0.063 0.000 0.009 0.071
2021-09-30 02:24:13,487 - mmdet - INFO - Epoch [4][10/245]	lr: 1.000e-02, eta: 4:55:36, time: 2.288, data_time: 0.344, memory: 14790, loss_cls: 1.2909, loss_bbox: 0.3953, loss_bbox_rf: 0.5065, loss: 2.1928, grad_norm: 6.0289
2021-09-30 02:24:33,350 - mmdet - INFO - Epoch [4][20/245]	lr: 1.000e-02, eta: 4:55:20, time: 1.986, data_time: 0.036, memory: 14790, loss_cls: 1.2389, loss_bbox: 0.3611, loss_bbox_rf: 0.4626, loss: 2.0626, grad_norm: 3.6556
2021-09-30 02:24:53,562 - mmdet - INFO - Epoch [4][30/245]	lr: 1.000e-02, eta: 4:55:08, time: 2.021, data_time: 0.038, memory: 14790, lo